In [ ]:
import numpy as np
import json
from livecell_tracker.core import (
    SingleCellTrajectory,
    SingleCellStatic,
    SingleCellTrajectoryCollection,
)
from livecell_tracker.segment.detectron_utils import (
    convert_detectron_instance_pred_masks_to_binary_masks,
    convert_detectron_instances_to_label_masks,
    segment_images_by_detectron,
    segment_single_img_by_detectron_wrapper,
)
from livecell_tracker.core.datasets import LiveCellImageDataset
from livecell_tracker.preprocess.utils import (
    overlay,
    enhance_contrast,
    normalize_img_to_uint8,
)
import matplotlib.pyplot as plt
import os
from tqdm import tqdm
from pathlib import Path
import pandas as pd

## Convert Labelme Json to COCO Json

run the following code once for generating coco json
```
import livecell_tracker.segment
import livecell_tracker.annotation
import livecell_tracker.annotation.labelme2coco
import os
labelme_json_folder = r"""../datasets/a549_ccnn/annotation_data"""
dataset_folder_path = r"""../datasets/a549_ccnn/original_data"""
export_dir = "./notebook_results/correction_cnn_v0.0.0/"
os.makedirs(export_dir, exist_ok=True)
livecell_tracker.annotation.labelme2coco.convert(
    labelme_json_folder,
    export_dir,
    train_split_rate=0.9,
    dataset_folder_path=dataset_folder_path,
    # is_image_in_json_folder=True,
    image_file_ext="tif",
    # image_file_ext="png",
)
```

## Load COCO into SingleCell Objects

In [ ]:
from pycocotools.coco import COCO

# coco_data = COCO("../datasets/a549_ccnn/a549_ccnn_coco_v0.0.0/train.json")
# out_dir = Path("./notebook_results/a549_ccp_vim/train_data_v8/")
coco_data = COCO("../datasets/a549_ccnn/a549_ccnn_coco_v0.0.0/val.json")
out_dir = Path("./notebook_results/a549_ccp_vim/test_data_v8/")

coco_data.anns.keys(), coco_data.anns[1].keys(), coco_data.anns[1]["segmentation"][0][
    :5
]


In [ ]:
coco_data.imgs.keys(), coco_data.imgs[1].keys(), coco_data.imgs[1]["file_name"],


In [ ]:
from typing import List

from livecell_tracker.annotation.coco_utils import coco_to_sc

single_cells = coco_to_sc(coco_data)


# # for testing
# single_cells = single_cells[:20]

fig, axes = plt.subplots(1, 3, figsize=(10, 5))
cell_id = 10
axes[0].imshow(single_cells[cell_id].get_img_crop(padding=100))
axes[1].imshow(single_cells[cell_id].get_contour_mask_closed_form(padding=100))
axes[2].imshow(single_cells[cell_id].get_contour_mask(padding=100))


In [ ]:
len(single_cells)

save a list of single cell objects

In [ ]:
SingleCellStatic.write_single_cells_json(
    single_cells, "../datasets/a549_ccnn/single_cells.json"
)


In [ ]:
contour_mask = single_cells[cell_id].get_contour_mask(padding=100)
contour_mask.astype(np.uint8)


In [ ]:
import cv2 as cv
from livecell_tracker.preprocess.utils import dilate_or_erode_mask

plt.imshow(dilate_or_erode_mask(contour_mask.astype(np.uint8), 1))


In [ ]:
sample_sc = single_cells[cell_id]
sample_sc.meta

In [ ]:
import glob

raw_img_dataset = sample_sc.img_dataset
seg_data_dir = "../datasets/a549_ccnn/seg_tiles_CCP_A549-VIM_lessThan24hr_Calcein_1mg-ml_DP_Ti2e_2022-9-11"
seg_paths = glob.glob(os.path.join(seg_data_dir, "*.png"))
print("sample seg paths:", seg_paths[:2])
matched_time2seg = {}
# for time, img_path in raw_img_dataset.time2url.items():
#     substr = os.path.basename(img_path).split(".")[0]
#     print("substr:", substr)
#     for seg_path
#     break
corrected_indices = []
for seg_path in seg_paths:
    substr = os.path.basename(seg_path).split(".")[0] # get rid of extension
    substr = substr[4:]  # get rid of seg_ prefix
    img, path, index = raw_img_dataset.get_img_by_url(
        substr, return_path_and_time=True, ignore_missing=True
    )
    if path is None:
        print("skip due to substr not found:", substr)
        continue
    matched_time2seg[index] = seg_path

crappy_seg_data = LiveCellImageDataset(time2url=matched_time2seg, ext="png")
sample_sc.mask_dataset = crappy_seg_data
assert len(crappy_seg_data) == len(raw_img_dataset)


## Generate Synthetic Undersegmentation data

In [ ]:
sc1 = single_cells[10]
sc2 = single_cells[1]

padding=20
fig, axes = plt.subplots(2, 3, figsize=(10, 5))
sc1.show(padding=padding, ax=axes[0][0])
sc1.show_contour_img(padding=padding, ax=axes[0][1])
sc1.show_contour_mask(padding=padding, ax=axes[0][2])
sc2.show(padding=padding, ax=axes[1][0])
sc2.show_contour_img(padding=padding, ax=axes[1][1])
sc2.show_contour_mask(padding=padding, ax=axes[1][2])

In [ ]:
from livecell_tracker.segment.ou_simulator import combine_two_scs_monte_carlo, compute_distance_by_contour,viz_check_combined_sc_result, gen_gauss_sc_bg, gen_sc_bg_crop, gen_synthetic_nonoverlap_scs, gen_synthetic_overlap_scs, gen_underseg_scs_sample, gen_aug_diff_mask, gen_underseg_scs, _gen_underseg_scs_sample_wrapper, compute_two_contours_min_distance, merge_two_scs_nonoverlap, merge_two_scs_overlap, move_util_in_range

combine two scs using monte carlo

In [ ]:
# bg_scale = 3.0

# for i in range(4):
#     new_sc1, new_sc2 = combine_two_scs_monte_carlo(sc1, sc2, bg_img=None)
#     viz_check_combined_sc_result(new_sc1, new_sc2)

# for i in range(4):
#     new_sc1, new_sc2 = combine_two_scs_monte_carlo(sc1, sc2, bg_img=None, fix_sc1=True)
#     viz_check_combined_sc_result(new_sc1, new_sc2)

In [ ]:
# viz_check_combined_sc_result(new_sc1, new_sc2)

check overseg synthesis

In [ ]:
# # check results
# for i in tqdm(range(4)):
#     new_sc1, new_sc2, _, is_success = gen_synthetic_overlap_scs(sc1, sc2)
#     viz_check_combined_sc_result(new_sc1, new_sc2)

### Generate synthetic non-overlapping underseg cells 

example: compute distance of two single cells

In [ ]:

sc3 = single_cells[3]
# fig, axes = plt.subplots(1, 3, figsize=(10, 5))
# sc1.show_contour_img(ax=axes[0], padding=10000)
# sc2.show_contour_img(ax=axes[1], padding=10000)
# sc3.show_contour_img(ax=axes[2], padding=10000)


compute_distance_by_contour(sc1, sc2), compute_distance_by_contour(sc1, sc3), compute_distance_by_contour(sc2, sc3)

example: generate a gaussian background

In [ ]:
plt.imshow(gen_gauss_sc_bg(sc1, (512, 512)))

example: generate a backgroudn according to bg

In [ ]:

plt.imshow(gen_sc_bg_crop(sc1, (512, 512)))

In [ ]:


# # check results
# min_dist = 30
# max_dist = 100
# for i in tqdm(range(2)):
#     new_sc1, new_sc2, _, is_success = gen_synthetic_nonoverlap_scs(sc1, sc2, gen_bg_func=gen_sc_bg_crop, min_dist=min_dist, max_dist=max_dist, dist_per_move=30)
#     viz_check_combined_sc_result(new_sc1, new_sc2)

# for i in tqdm(range(2)):
#     new_sc1, new_sc2, _, is_success = gen_synthetic_nonoverlap_scs(sc1, sc2, gen_bg_func=gen_gauss_sc_bg, min_dist=min_dist, max_dist=max_dist, dist_per_move=30)
#     viz_check_combined_sc_result(new_sc1, new_sc2)

example: generate nonoverlap scs and move them together

In [ ]:
import random
# for i in tqdm(range(20)):
#     # randomly choose sc1 and sc2
#     sc1, sc2 = random.sample(single_cells, 2)
#     new_sc1, new_sc2, _, is_success = gen_synthetic_nonoverlap_scs(sc1, sc2, gen_bg_func=gen_gauss_sc_bg, min_dist=10, max_dist=1000,  min_reserved_area_percent=1)
#     viz_check_combined_sc_result(new_sc1, new_sc2)
#     # move_two_syn_scs_close_or_apart(new_sc1, new_sc2, dist=40, bg_img=gen_gauss_sc_bg(new_sc1, shape=new_sc1.get_img().shape), inplace=True)
#     move_util_in_range(new_sc1, new_sc2, dist_per_move=30, bg_img=gen_gauss_sc_bg(new_sc1, shape=new_sc1.get_img().shape), min_dist=10, max_dist=40, inplace=True, max_move=100)
#     viz_check_combined_sc_result(new_sc1, new_sc2)

In [ ]:
from typing import Tuple
from skimage.measure import regionprops
import pandas as pd
from livecell_tracker.core.io_utils import save_tiff
from livecell_tracker.segment.ou_utils import csn_augment_helper, underseg_overlay_gt_masks, gen_aug_diff_mask


check sc areas 
```
for sc in single_cells:
    prop = sc.compute_regionprops()
    assert prop.area > 5
```

In [ ]:
import itertools
from skimage.measure import find_contours
import cv2

Viz merge two cells

Overlap case

In [ ]:
new_sc1, new_sc2, _, _ = gen_synthetic_overlap_scs(sc1, sc2, min_reserved_area_percent=1)
sc3, is_success = merge_two_scs_overlap(new_sc1, new_sc2)
if is_success:
    sc3.show()

Nonoverlap case

In [ ]:
for i in range(1):
    sc1, sc2 = random.sample(single_cells, 2)
    new_sc1, new_sc2, _, _ = gen_synthetic_nonoverlap_scs(sc1, sc2, gen_bg_func=gen_gauss_sc_bg, min_dist=10, max_dist=50, bg_scale=2, min_reserved_area_percent=1)
    sc3, is_success = merge_two_scs_nonoverlap(new_sc1, new_sc2)
    if is_success:
        sc3.show()
        plt.show()

show nonoverlap merge case: merge by dilation. sc3 represents a generated under-segmentation case.

In [ ]:
new_sc1, new_sc2, _, _ = gen_synthetic_nonoverlap_scs(sc1, sc2, gen_bg_func=gen_sc_bg_crop)
sc3, is_success = merge_two_scs_nonoverlap(new_sc1, new_sc2)
if is_success:
    fig, axes = plt.subplots(1, 4)
    padding = 100
    new_sc1.show_mask(ax=axes[0], crop=True, padding=padding)
    axes[0].set_title("sc1")
    new_sc2.show_mask(ax=axes[1], crop=True, padding=padding)
    axes[1].set_title("sc2")
    sc3.show_mask(ax=axes[2], crop=True, padding=padding)
    axes[2].set_title("sc3")

    axes[3].imshow(new_sc1.get_mask() + new_sc2.get_mask())


In [ ]:
def test_gen_underseg_scs():
    subdir = Path("synthetic_underseg_overlap")
    underseg_out_dir = out_dir / subdir
    gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=0, num_cells=4, total_sample_num=1)
    gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=1, num_cells=4, total_sample_num=1)

def gen_syn_overlap_underseg_data_v8(sample_num=10):
    subdir = Path("synthetic_underseg_overlap")
    underseg_out_dir = out_dir / subdir
    # generate training data
    gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=0, num_cells=4, total_sample_num=sample_num)
    gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=sample_num, num_cells=3, total_sample_num=sample_num)
    gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=sample_num * 2, num_cells=2, total_sample_num=sample_num * 2)


def gen_syn_overlap_underseg_test_data_v8(sample_num = 100):
    def _save_all_df(all_df):
        all_df.to_csv(out_dir / "synthetic_underseg_overlap" / "data.csv", index=False)

    subdir = Path("synthetic_underseg_overlap")
    underseg_out_dir = out_dir / subdir
    # generate training data
    all_df = gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=0, num_cells=4, total_sample_num=sample_num)
    _save_all_df(all_df)
    df = gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=sample_num, num_cells=3, total_sample_num=sample_num)
    all_df = pd.concat([all_df, df], ignore_index=True)
    _save_all_df(all_df)
    df = gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=sample_num * 2, num_cells=2, total_sample_num=sample_num * 2)
    all_df = pd.concat([all_df, df], ignore_index=True)
    _save_all_df(all_df)



def gen_syn_nonoverlap_underseg_train_data(gen_bg_func, subdir, sample_num = 10):
    underseg_out_dir = out_dir / subdir
    def _save_all_df(all_df):
        all_df.to_csv(underseg_out_dir / "data.csv", index=False)

    # generate training data    
    generator_args = {
        "min_dist": 3,
        "max_dist": 20,
        "bg_scale": 3,
        "gen_bg_func": gen_bg_func
    }
    all_df = gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=0, num_cells=4, total_sample_num=sample_num, sc_generator_func=gen_synthetic_nonoverlap_scs, sc_generator_func_kwargs=generator_args, merge_func=merge_two_scs_nonoverlap)
    _save_all_df(all_df)
    df = gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=sample_num, num_cells=3, total_sample_num=sample_num, sc_generator_func=gen_synthetic_nonoverlap_scs, sc_generator_func_kwargs=generator_args, merge_func=merge_two_scs_nonoverlap)
    all_df = pd.concat([all_df, df], ignore_index=True)
    _save_all_df(all_df)
    df = gen_underseg_scs(scs=single_cells, save_dir=underseg_out_dir, sample_id_offset=sample_num * 2, num_cells=2, total_sample_num=sample_num * 2, sc_generator_func=gen_synthetic_nonoverlap_scs, sc_generator_func_kwargs=generator_args, merge_func=merge_two_scs_nonoverlap)
    all_df = pd.concat([all_df, df], ignore_index=True)
    _save_all_df(all_df)


Single core version for testing

check if each single cell has mask ground truth information

In [ ]:
for sc in single_cells:
    assert (sc.mask_dataset is not None) or (sc.contour is not None)

In [ ]:
# for i in range(50):
#     tmp_scs = random.sample(single_cells, 3)
#     gen_underseg_scs_sample(scs=tmp_scs, num_cells=3, viz_check=True, sc_generator_func=gen_synthetic_nonoverlap_scs, sc_generator_func_kwargs={"min_dist": 1, "max_dist": 20, "bg_scale": 3, "gen_bg_func": gen_gauss_sc_bg}, merge_func=merge_two_scs_nonoverlap)

In [ ]:
def test_gen():
    subdir = Path("synthetic_underseg_nonoverlap_test")
    underseg_out_dir = out_dir / subdir
    generator_args = {"min_dist": 10, "max_dist": 30, "bg_scale": 3, "gen_bg_func": gen_gauss_sc_bg}
    df = gen_underseg_scs(
        scs=single_cells,
        save_dir=underseg_out_dir,
        sample_id_offset=0,
        num_cells=3,
        total_sample_num=50,
        sc_generator_func=gen_synthetic_nonoverlap_scs,
        sc_generator_func_kwargs=generator_args,
        merge_func=merge_two_scs_nonoverlap,
        viz_check=True,
        parallel=False,
    )


In [ ]:
# test_gen()

In [ ]:
# train data
# gen_syn_nonoverlap_underseg_train_data(gen_bg_func=gen_gauss_sc_bg, subdir=Path("synthetic_underseg_nonoverlap_gauss"), sample_num=500);

# test data
gen_syn_nonoverlap_underseg_train_data(gen_bg_func=gen_gauss_sc_bg, subdir=Path("synthetic_underseg_nonoverlap_gauss"), sample_num=100)

In [ ]:
# gen_syn_underseg_data_v8(sample_num=1000)
# gen_syn_overlap_underseg_test_data_v8()

In [ ]:
# gen_underseg_scs(scs=single_cells, total_sample_num=5, save_dir=None, num_cells=3, viz_check=True)

In [ ]:
# gen_underseg_scs(scs=single_cells, total_sample_num=5, save_dir=None, num_cells=2, viz_check=True)

In [ ]:
dataframes = []
for subdir in out_dir.iterdir():
    if subdir.is_dir():
        data_path = subdir / "data.csv"
        dataframe = pd.read_csv(data_path)
        dataframe["subdir"] = subdir.name
        dataframes.append(dataframe)
combined_dataframe = pd.concat(dataframes)
combined_dataframe.to_csv(out_dir / "train_data.csv", index=False)

Now we need to handle two cases:
1) there is any overlap between two objects  
    a) simply dilate and create underseg cases  
2) there is no overlap (future work) 
    b) fill in the pixels in-between the two objects  
    c) 

## Combine data.csv files generated in each subfolder

In [ ]:
dataframes = []
for subdir in out_dir.iterdir():
    if subdir.is_dir():
        data_path = subdir / "data.csv"
        dataframe = pd.read_csv(data_path)
        dataframe["subdir"] = subdir.name
        dataframes.append(dataframe)
combined_dataframe = pd.concat(dataframes)
combined_dataframe.to_csv(out_dir / "train_data.csv", index=False)

In [ ]:
len(combined_dataframe)